<a href="https://colab.research.google.com/github/Newbie-Nyx/simple-img2img/blob/main/%E3%82%B7%E3%83%B3%E3%83%97%E3%83%AB%E3%81%AAStable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q diffusers==0.2.4
!pip install -q transformers scipy ftfy
!pip install -q "ipywidgets>=7,<8"

#@markdown 右のURLに飛んでご自身のトークンを取得し、下に貼り付けてください。 (https://huggingface.co/settings/tokens)
YOUR_TOKEN=""#@param {type:"string"}
#@markdown　一度張り付けた後にセーブ（Ctrl + s）すれば、２回目以降はURL先に飛ばなくても残っていると思います。なので、2回目以降は実行ボタンを押すだけで良いはずです

import torch
from diffusers import StableDiffusionPipeline
from torch import autocast

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=YOUR_TOKEN) 
pipe.to("cuda")

# Stablle Diffusion

In [ ]:
from google.colab import files        #@title Text to Image
from torch.random import seed
from torch import autocast 
from pathlib import Path
import random
import shutil
import os
#-----------------------------------------------------------------------------------------
# NSFWの制限を外す
pipe.safety_checker = lambda images, **kwargs: (images, False)
#-----------------------------------------------------------------------------------------
try:
    shutil.rmtree('/A')               #フォルダAを消す
    os.remove('/AIart.zip')           #zipファイルを消す
except FileNotFoundError:
    pass
#-----------------------------------------------------------------------------------------
path = '/A'                           #フォルダAを作成
os.makedirs(path, exist_ok=True)
#-----------------------------------------------------------------------------------------
i = 50 #@param {type:"slider", min:1, max:50, step:1}
for I in range(i):

  x = random.randint(100000000, 999999999)
  generator = torch.Generator("cuda").manual_seed(x)

  prompt = ""#@param {type:"string"}

  with autocast("cuda"):
    height = 512 #@param {type:"slider", min:512, max:960, step:64}
    width  = 768 #@param {type:"slider", min:512, max:960, step:64}
    guidance_scale = 10#@param {type:"slider", min:1, max:20, step:0.1}
    Step  = 50 #@param {type:"slider", min:1, max:250, step:1}
    image = pipe(prompt, height, width, guidance_scale=7.5, num_inference_steps=Step, generator=generator)["sample"][0]

    image.save(f"/A/{x}.png")
#-----------------------------------------------------------------------------------------
if i == 1:
  files.download(f"/A/{x}.png")
else:
  !zip -r /AIart.zip /A                   # フォルダAを zip 圧縮する

  from google.colab import files          # 圧縮したzipファイルをダウンロードする
  files.download("/AIart.zip")

#-----------------------------------------------------------------------------------------
